### Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

### Чтение файла

In [2]:
df = pd.read_csv('corp_lemm.csv')

In [3]:
df = df[df['text'].notnull()]
df = df.drop(['Unnamed: 0'], axis = 1)

In [4]:
df.head(6)

,text,rated,coloring,date,name,bank,sex,text_lemmatized
0,Возмутительный способ обмана придумали крючкот...,1,bad,"04 апр, 2022",Сергей,vtb,male,возмутительный способ обман придумать крючкотв...
1,"Не храните в нем деньги, ни коем случае! У мен...",1,bad,"30 мар, 2022",Альбина,uralsib,female,не хранить немой деньга кой случай у карта сч...
2,С 09.03.22 не могу получить средства с евро сч...,1,bad,"24 мар, 2022",Ирина,uralsib,female,с мочь получить средство евро счёт должный выд...
3,"Очень долгое обслуживание, сотрудники совершен...",1,bad,"14 мар, 2022",Александр,gazprombank,male,очень долгий обслуживание сотрудник совершенно...
4,Звоню третий день по вопросу валюты. Оператор ...,1,bad,"14 мар, 2022",Сергей Владимирович,gazprombank,unkn,звонить третий день вопрос валюта оператор го...
5,"Опыт сложился негативный, Сегодня 3марта 2022 ...",1,bad,"03 мар, 2022",Илья,gazprombank,male,опыт сложиться негативный сегодня март год сра...


### Анализ тональности

In [5]:
#присвоение целевого признака
sentiment_mapping = {'bad': 0, 'neutral': 0, 'good': 1}
df['label'] = df['coloring'].map(sentiment_mapping)
df['label'].value_counts()

1    2120
0    1996
Name: label, dtype: int64

In [6]:
#разбиение на обучающую и тестовую выборки
train, test = train_test_split(df,
                        test_size = 0.2,
                        random_state = 12348,
                       )

print(train.shape)
print(test.shape)

(3292, 9)
(824, 9)


In [7]:
#распределение целевого признака
for sample in [train, test]:    
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

0.5212636695018226
0.49029126213592233


In [8]:
nlp1 = [i for i in train['text_lemmatized']]
nlp2 = [i for i in test['text_lemmatized']]

In [9]:
#векторизация
tfidf = TfidfVectorizer(ngram_range=(1,1))

In [10]:
tfidf_train_base = tfidf.fit_transform(train['text_lemmatized'])

In [11]:
tfidf_test_base = tfidf.transform(test['text_lemmatized'])

In [12]:
tfidf_tokens = tfidf.get_feature_names()

In [13]:
df_tfidfvect = pd.DataFrame(data = tfidf_test_base.toarray(),columns = tfidf_tokens)

In [14]:
tfidf_train_base.shape

(3292, 12705)

In [15]:
tfidf_test_base.shape

(824, 12705)

In [16]:
#обучение модели
clf = LogisticRegression(multi_class='ovr', class_weight='balanced', max_iter=10000)
clf.fit(tfidf_train_base, train['label'])
pred_train, pred_test = clf.predict_proba(tfidf_train_base), clf.predict_proba(tfidf_test_base)

In [17]:
test['predictions'] = [x.round(3).tolist() for x in pred_test]
pred_train_label, pred_test_label = pred_train.argmax(axis=1), pred_test.argmax(axis=1)

<ipython-input-17-bf47405a233f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predictions'] = [x.round(3).tolist() for x in pred_test]


### Оценка качества работы алгоритма

In [18]:
def multiclass_roc_auc(y_true, y_pred):
    if len(set(y_true)) == 2:
        r = roc_auc_score(y_true, y_pred[:, 1])
    else:
        r = roc_auc_score(y_true, y_pred, multi_class='ovr')
    return r
print('ROC-AUC:\n')
print(multiclass_roc_auc(train['label'], pred_train))
print(multiclass_roc_auc(test['label'], pred_test))

ROC-AUC:

0.9836397211079952
0.9616513437057991


In [19]:
pred = []
for i in test['predictions']:
    if i[0]>i[1]:
        pred.append('neg')
    else:
        pred.append('pos')
tr = []
for i in test['coloring']:
    if i == 'good':
        tr.append('pos')
    else:
        tr.append('neg')
test = test.drop(['coloring', 'text_lemmatized','date','name','bank','sex'], axis=1)
test['true'] = tr
test['predicted'] = pred

In [20]:
a = []
b = []
for i in test['predicted']:
    a.append(i)
for i in test['true']:
    b.append(i)

In [21]:
tp = 0 
tn = 0 
fp = 0 
fn = 0
t_f = []
for i in range(0, len(a)):
    if a[i] == b[i]:
        if a[i] == 'pos':
            tp+=1
            t_f.append('tp')
        else:
            tn+=1
            t_f.append('tn')
    elif a[i] != b[i]:
        if a[i] == 'pos':
            fp+=1
            t_f.append('fp')
        else:
            fn+=1
            t_f.append('fn')
        

In [22]:
test['result'] = t_f

In [23]:
accuracy = (tp+tn)/len(test)
precision = tp/ (tp+fp)
recall = tp/(tp+fn)
f1 = 2/(1/precision+1/recall)
f_measure = 2*((precision*recall)/(precision+recall))

In [24]:
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)
print('f1-score', f1)
print('f-measure', f_measure)

accuracy 0.9150485436893204
precision 0.946524064171123
recall 0.8762376237623762
f1-score 0.910025706940874
f-measure 0.9100257069408739


In [25]:
t_res = test.query('result == "tn" or result == "tp"')
f_res = test.query('result == "fn" or result == "fp"')
pd.options.display.max_colwidth = 10000

In [26]:
f_res.query('result == "fn"') #ложноотрицательные отзывы

,text,rated,label,predictions,true,predicted,result
2680,"Пришла в 17.15 -ведется запись, мой номер 76, к этому времени прошло 35 человек, шансов до 20.00 почти нет. Ладно, приду в субботу пораньше. Сегодня пришла в 9.10 утра, у дверей толпа человек 12. Пустили в офис ровно в 10.00 Меня удивило, что в субботу (короткий день,работают до 16.00) работали все 5 окон, никаких скандалов из-за очереди, я была где-то 15-я и оформила все до 11 утра. Никаких раздражительных сотрудников, девочки работали оперативно и вежливо. Состояние авторов предыдущих негативных отзывов понять могу. Сама была не раз в такой ситуации.Но ведь иногда мы же сами такую ситуацию создаем. Сейчас в очереди в основном пенсионеры сами нагнетают нервозность, слухи пускают. А ведь в банке работают обычные люди, такие же как наши дети или близкие. Не все могут сдержать раздражение или агрессию при таком наплыве людей. Поймите меня правильно, я ни в коем случае не оправдываю грубость и хамство, просто иногда надо встать на место другого. Сейчас сама не могу получить досрочно закрытый вклад в другом банке, тоже на грани срыва, но подумала что тратить свое здоровье из-за этих денег не стоит. Поэтому сегодняшнее посещение отделения на Садовнической ул. оставило хорошее впечатление.",4,1,"[0.869, 0.131]",pos,neg,fn
2511,"Самый плохой банк. Справедливости ради надо сказать: почти всегда в офисах нет или почти нет очереди. Кредит ( при небольших танцах с бубном) можно взять не выходя из дома. Но за все это придется платить- только узнаешь об этом поздно. Брал кредит. Навязали страховку. Брал на 4 года. Через 6 месяцев решил погасить досрочно. Нужна была справка для рефинансирования. Бесплатную оказывается надо ждать 20 дней!!! Если надо в течении дня- 300 рублей. В течении 4 часов- 500 рублей. В двух других банках дали такую справку СРАЗУ и БЕСПЛАТНО. А вернуть часть страховки за кредит просто не реально. Оказывается я могу расторгнуть страховку, но ничего не вернут. А ведь прошло всего полгода. В другом банке, при выдаче БЕСПЛАТНОЙ справки сразу же спросили- буду ли я расторгать договор страхования кредита, рассказали как это сделать без проволочек и какую сумму мне вернут. Хочет все больше и больше отобрать у граждан, вынужденных в этот банк обратиться, честно заработанное. После праздников напишу заявление своему работодателю о переводе моей зарплаты в НАСТОЯЩИЙ банк.",4,1,"[0.904, 0.096]",pos,neg,fn
2600,"Оформляю кредит в Россельхозбанке (Мурманск)под залог имущества уже пятый месяц...как говориться курам на смех ))) вообще и надобность в кредите пропала,только время потерял и работу по сбору бумаг проделали впустую! Интересно, у всех так? В Россельхозбанке есть какая либо отчетность по профессиональной оценке работы специалистов или нет?",4,1,"[0.579, 0.421]",pos,neg,fn
2487,"Получил долгожданную ипотеку в Совкомбанке на выгодных условиях. Подавал заявки сразу в несколько банков, здесь мне ответили первыми, одобрили, пригласили в офис с документами. При оформлении менеджер Константин обратил внимание на мою справку 2 НДФЛ, как оказалось, она была оформлена с ошибкой, печать стояла не в левом, а в правом углу. Казалось бы, мелочь, но нужно исправлять. Я сразу же позвонил в бухгалтерию и попросил переоформить справку, получил ее на следующий день и привез в банк. Ипотеку одобрили, договор подписали уже через две недели, все сложилось удачно. Константину спасибо за внимательность и вежливость, работой специалиста я очень доволен. А балл снимаю за то, что при расчете ипотеки онлайн на сайте сумма одна, а одобрили в результате сумму меньше, не критично, но все-таки. Но это уже проблема скорее самого сайта.",4,1,"[0.544, 0.456]",pos,neg,fn
2288,"За несколько последних лет обслуживания отмечаю в последнее время (месяцев 6) некоторое ухудшение качества обслуживания на линии технической поддержки в чате приложения с точки зрения понимания проблемы, с которой я обращаюсь. Пример, выполняю условия получения дополнительного кешбэка, купить заемными средствами 30 единиц товара,

In [27]:
f_res.query('result == "fp"') #ложноположительные отзывы

,text,rated,label,predictions,true,predicted,result
1339,"января текущего года в мобильном приложении не увидела начисленных процентов по накопительному счету ""Управляй процентом"" за прошедший месяц. Ответного сигнала от представителей банка пока не получила, равно как и потерянной суммы. Аналогичная ситуация сложилась и у мамы, имеющей такой же счёт. Поиски продолжаются. Весьма печально и неожиданно, до настоящего момента претензий к обслуживанию банка не было.",3,0,"[0.444, 0.556]",neg,pos,fp
1330,"Много клиентов, мало сотрудников, долгое время ожидания. Сотрудники вежливые, доброжелательные. Офис небольшой.",3,0,"[0.256, 0.744]",neg,pos,fp
1989,Иногда в зале обслуживает меньше одного работника. По моему это не очень хороший показатель. Услуг тоже не много.,3,0,"[0.319, 0.681]",neg,pos,fp
85,"Повелся на ""бесплатное обслуживание"" за разовый платеж, хотя все отлично работало в другом банке. Менеджер убедил, что запасной р/с всегда хорошо иметь. Но в бесплатном обслуживании расчетного счета автоматически зашито платное обслуживание корп карты, даже если по ней не было движений. Об этом менеджер конечно же забыл упомянуть.",1,0,"[0.324, 0.676]",neg,pos,fp
539,год получаю сообщения от банка с 4 до 5 ночи.очень приято!!!УР........ДЫ из банка говорят что это нормально.....?????,1,0,"[0.434, 0.566]",neg,pos,fp
1210,"г. я хотел открыть вклад в долларах на приличную сумму, но так и ушел ни с чем. Потерял больше часа. Операторы работают очень медленно, недружелюбны, на вопрос о процентной ставке по вкладам ответили вопросом на вопрос. ""откуда мы их помним, они часто меняются........ждите"". Ну настоящее СЕЛЬПО.",2,0,"[0.432, 0.568]",neg,pos,fp
1819,"Пользуюсь рокетбанком около двух лет и всё было на нормальном уровне до перехода к qiwi. Единственное неудобство, что за рокет рубли (кэшбэк) можно только свои покупки компенсировать и в ограниченном диапазоне цен. Но последнее время творится что-то возмутительное! При оплате налогов приложение висло, причём в разных местах или не давало оправить плажёт. Заполнять в мобильном приложении 10 полей и так не самое приятное занятие, а когда по вине приложения платёж еще и не проходит - совсем плохо. Техподдержка сначала предлагала попробовать всякие хитрые обходные пути - заполнить сначала последние поля, потом первые и т.д. Они знали о проблеме в приложении и обещали написать мне в течение 2-х суток. Оформить самим плажёт от моего имени они отказались. Задача оплаты подвисла и сейчас уже висит неделю! Сначала поддержка отвечала с задержкой в полдня, а последние 2 дня поддержка банка не выходит на связь совсем. На звонок по телефону отвечает робот, предлагает писать в чат и вешает трубку. Я звонил несколько раз - голосового человеческого оператора нет. Итого: офиса нет, телефонной поддержки нет, поддержка в приложении не отвечает днями.",3,0,"[0.393, 0.607]",neg,pos,fp
1400,Отвратительный банк !никогда больше с ним не свяжусь был автокредит в быстро банке потом перенаправили в экспобанк не оповести ! Просто 4месяца и опять перевели в быстро банк не оповестив !!никогда не берите так кредит !НИКОГДА,3,0,"[0.359, 0.641]",neg,pos,fp
381,"В мобильном приложении банка есть «кредитный рейтинг». Нажал «рассчитать» - написали «очень низкий». Обратился в БКИ - рейтинг «высокий». Написал в банк в чат - говорят что это БКИ им такой рейтинг предоставил, откровенно лгут.",1,0,"[0.369, 0.631]",neg,pos,fp
1898,"Мое мнение о Росбанке достаточно нейтральное. Есть в его работе как некоторые недочеты, так и положительные моменты. Если начать с плохого - то совершенно не радует то, что временами деньги, которые переводят на карту этого банка идут слишком долго. Нет, такого, что бы они вообще куда-то пропали не было, однако временами нужно моментально их использовать, а их все нет. Из положительного - нравится то, что есть большое количество банкоматов, в том числе и в трех минутах ходьбы от дома. Персонал у них работает обычный, есть приятные ребята, а есть явно недовольные всем особи, которые не прочь и нахами

### Общие выводы

Модель лучше справилась с определением тональности на обучающем наборе данных, что ожидаемо, но потеря качества небольшая.

Наблюдаем высокие показатели по всем метрикам, но они ниже, чем полученные при анализе тональности методом, основанном на правилах.

Мы предположили, что использование разных словоформ в тексте может влиять на эмоциональную окраску, и при лемматизации результат искажается. Поэтому анализ тональности тем же алгоритмом был проведен еще раз, но модель векторизации была обучена на сырых текстах. Получили следующие значения метрик:

Roc-auc(train) 0.9840383284228462
Roc-auc(test) 0.9557696841112683
accuracy 0.9114077669902912
precision 0.9366754617414248
recall 0.8787128712871287
f1-score 0.9067688378033205
f-measure 0.9067688378033205

Наша гипотеза не подтвердилась: результаты работы алгоритма по всем использованным метрикам на лемматизированных текстах выше, чем на необработанных.
